# Practical Exam: Supermarket Loyalty

International Essentials is an international supermarket chain.

Shoppers at their supermarkets can sign up for a loyalty program that provides rewards each year to customers based on their spending. The more you spend the bigger the rewards. 

The supermarket would like to be able to predict the likely amount customers in the program will spend, so they can estimate the cost of the rewards. 

This will help them to predict the likely profit at the end of the year.

## Data

The dataset contains records of customers for their last full year of the loyalty program.

| Column Name | Criteria                                                |
|-------------|---------------------------------------------------------|
|customer_id | Unique identifier for the customer. </br>Missing values are not possible due to the database structure. |
|spend | Continuous. </br>The total spend of the customer in their last full year. This can be any positive value to two decimal places. </br>Missing values should be replaced with 0. |
|first_month | Continuous. </br>The amount spent by the customer in their first month of the year. This can be any positive value, rounded to two decimal places. </br>Missing values should be replaced with 0. |
| items_in_first_month | Discrete. </br>The number of items purchased in the first month. Any integer value greater than or equal to zero. </br>Missing values should be replaced by 0. |  
| region | Nominal. </br>The geographic region that the customer is based in. One of four values Americas, Asia/Pacific, Europe, Middle East/Africa. </br>Missing values should be replaced with "Unknown". |
| loyalty_years | Oridinal. </br>The number of years the customer has been a part of the loyalty program. One of five ordered categories, '0-1', '1-3', '3-5', '5-10', '10+'. </br>Missing values should be replaced with '0-1'.|
| joining_month | Nominal. </br>The month the customer joined the loyalty program. One of 12 values "Jan", "Feb", "Mar", "Apr", etc. </br>Missing values should be replaced with "Unknown".|
| promotion | Nominal. </br>Did the customer join the loyalty program as part of a promotion? Either 'Yes' or 'No'. </br>Missing values should be replaced with 'No'.|


In [1]:
# ✅ Import libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor

# Task 1

Before you fit any models, you will need to make sure the data is clean. 

The table below shows what the data should look like. 

Create a cleaned version of the dataframe. 

 - You should start with the data in the file "loyalty.csv". 

 - Your output should be a dataframe named `clean_data`. 

 - All column names and values should match the table below.

| Column Name | Criteria                                                |
|-------------|---------------------------------------------------------|
|customer_id | Unique identifier for the customer. </br>Missing values are not possible due to the database structure. |
|spend | Continuous. </br>The total spend of the customer in their last full year. This can be any positive value to two decimal places. </br>Missing values should be replaced with 0. |
|first_month | Continuous. </br>The amount spent by the customer in their first month of the year. This can be any positive value, rounded to two decimal places. </br>Missing values should be replaced with 0. |
| items_in_first_month | Discrete. </br>The number of items purchased in the first month. Any integer value greater than or equal to zero. </br>Missing values should be replaced by 0. |  
| region | Nominal. </br>The geographic region that the customer is based in. One of four values Americas, Asia/Pacific, Europe, Middle East/Africa. </br>Missing values should be replaced with "Unknown". |
| loyalty_years | Oridinal. </br>The number of years the customer has been a part of the loyalty program. One of five ordered categories, '0-1', '1-3', '3-5', '5-10', '10+'. </br>Missing values should be replaced with '0-1'.|
| joining_month | Nominal. </br>The month the customer joined the loyalty program. One of 12 values "Jan", "Feb", "Mar", "Apr", etc. </br>Missing values should be replaced with "Unknown".|
| promotion | Nominal. </br>Did the customer join the loyalty program as part of a promotion? Either 'Yes' or 'No'. </br>Missing values should be replaced with 'No'.|

In [2]:
# Use this cell to write your code for Task 1
# ============================================
# ✅ Task 1: Identify, replace, and clean data
# ============================================

import pandas as pd
import numpy as np

# Load dataset
loyalty = pd.read_csv('loyalty.csv')

# Copy to preserve original
clean_data = loyalty.copy()

# ----- NUMERIC CLEANING -----
# Convert spend and first_month to numeric safely
for col in ['spend', 'first_month']:
    clean_data[col] = pd.to_numeric(clean_data[col], errors='coerce')
    clean_data[col] = clean_data[col].fillna(0)

# items_in_first_month should be integer
clean_data['items_in_first_month'] = pd.to_numeric(
    clean_data['items_in_first_month'], errors='coerce'
).fillna(0).astype(int)

# ----- CATEGORICAL CLEANING -----
# Fill missing categorical values
clean_data['region'] = clean_data['region'].fillna('Unknown')
clean_data['loyalty_years'] = clean_data['loyalty_years'].fillna('0-1')
clean_data['joining_month'] = clean_data['joining_month'].fillna('Unknown')
clean_data['promotion'] = clean_data['promotion'].fillna('No')

# ----- STRING MANIPULATION -----
# Remove leading/trailing spaces and standardize capitalization
for col in ['region', 'loyalty_years', 'joining_month', 'promotion']:
    clean_data[col] = clean_data[col].astype(str).str.strip().str.title()

# Fix known inconsistencies manually (optional but good practice)
clean_data['region'] = clean_data['region'].replace({
    'Asia/pacific': 'Asia/Pacific',
    'Middle east/africa': 'Middle East/Africa'
})

# ----- FINAL CHECK -----


# Task 2 

The team at International Essentials have told you that they have always believed that the number of years in the loyalty scheme is the biggest driver of spend. 

Producing a table showing the difference in the average spend by number of years in the loyalty programme along with the variance to investigate this question for the team.

 - You should start with the data in the file 'loyalty.csv'.

 - Your output should be a data frame named `spend_by_years`. 

 - It should include the three columns `loyalty_years`, `avg_spend`, `var_spend`. 

 - Your answers should be rounded to 2 decimal places.   

In [3]:
# Use this cell to write your code for Task 2
# ============================================
# ✅ Task 2: Group and summarize spend data
# ============================================

# Reload or reuse cleaned data
loyalty = pd.read_csv('loyalty.csv')

# Fill minimal missing values
loyalty['spend'] = pd.to_numeric(loyalty['spend'], errors='coerce').fillna(0)
loyalty['loyalty_years'] = loyalty['loyalty_years'].fillna('0-1')

# Group by loyalty_years and calculate mean & variance
spend_by_years = (
    loyalty.groupby('loyalty_years')['spend']
    .agg(avg_spend='mean', var_spend='var')
    .reset_index()
)

# Round to 2 decimal places
spend_by_years['avg_spend'] = spend_by_years['avg_spend'].round(2)
spend_by_years['var_spend'] = spend_by_years['var_spend'].round(2)

print(spend_by_years)


  loyalty_years  avg_spend  var_spend
0           0-1     110.56       9.30
1           1-3     129.31       9.65
2           10+     117.41      16.72
3           3-5     124.55      11.09
4          5-10     135.15      14.10


# Task 3

Fit a baseline model to predict the spend over the year for each customer.

 1. Fit your model using the data contained in “train.csv” </br></br>

 2. Use “test.csv” to predict new values based on your model. You must return a dataframe named `base_result`, that includes `customer_id` and `spend`. The `spend` column must be your predicted values.

In [4]:
# Use this cell to write your code for Task 3
# ============================================
# ✅ Task 3: Baseline Model (Linear Regression)
# ============================================

from sklearn.linear_model import LinearRegression

# Load train and test data
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

# Fill missing values similar to Task 1
for df in [train, test]:
    if 'spend' in df.columns:
        df['spend'] = pd.to_numeric(df['spend'], errors='coerce').fillna(0)
    df['first_month'] = pd.to_numeric(df['first_month'], errors='coerce').fillna(0)
    df['items_in_first_month'] = pd.to_numeric(df['items_in_first_month'], errors='coerce').fillna(0).astype(int)
    df['region'] = df['region'].fillna('Unknown')
    df['loyalty_years'] = df['loyalty_years'].fillna('0-1')
    df['joining_month'] = df['joining_month'].fillna('Unknown')
    df['promotion'] = df['promotion'].fillna('No')

# Encode categorical variables
X_train = pd.get_dummies(train.drop(['customer_id', 'spend'], axis=1), drop_first=True)
y_train = train['spend']

X_test = pd.get_dummies(test.drop(['customer_id'], axis=1), drop_first=True)
X_test = X_test.reindex(columns=X_train.columns, fill_value=0)

# Fit linear regression model
base_model = LinearRegression()
base_model.fit(X_train, y_train)

# Predict
base_predictions = base_model.predict(X_test)

# Create result dataframe
base_result = pd.DataFrame({
    'customer_id': test['customer_id'],
    'spend': base_predictions
})

print(base_result.head())


   customer_id       spend
0            5  140.699644
1            7  148.730919
2           16  140.810384
3           17  150.649670
4           19  153.628831


# Task 4

Fit a comparison model to predict the spend over the year for each customer.

 1. Fit your model using the data contained in “train.csv” </br></br>

 2. Use “test.csv” to predict new values based on your model. You must return a dataframe named `compare_result`, that includes `customer_id` and `spend`. The `spend` column must be your predicted values.

In [5]:
# Use this cell to write your code for Task 4
# ============================================
# ✅ Task 4: Comparison Model (Random Forest)
# ============================================

from sklearn.ensemble import RandomForestRegressor

# Fit Random Forest Regressor
rf_model = RandomForestRegressor(
    n_estimators=100, random_state=42, max_depth=10
)
rf_model.fit(X_train, y_train)

# Predict on test data
rf_predictions = rf_model.predict(X_test)

# Create comparison result dataframe
compare_result = pd.DataFrame({
    'customer_id': test['customer_id'],
    'spend': rf_predictions
})

print(compare_result.head())


   customer_id       spend
0            5  140.591910
1            7  148.739854
2           16  141.101224
3           17  150.831035
4           19  153.627259


⚙️ Optional (For You to Validate Before Submission)

You can check your RMSE on training data (for model performance before grading):

In [6]:
from sklearn.metrics import mean_squared_error
import numpy as np

train_preds = rf_model.predict(X_train)
rmse = np.sqrt(mean_squared_error(y_train, train_preds))
print("Training RMSE:", round(rmse, 3))


Training RMSE: 0.206
